# Explaining Knowledge Graphs with LLM

In [ ]:
%%capture

! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.26
! wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf?download=true
! mv llama-2-7b-chat.Q4_K_M.gguf?download=true ./llama-2-7b-chat.Q4_K_M.gguf
! pip install rdflib==7.0.0

## Main class for running LLama

In [ ]:
from llama_cpp import Llama
import os
import re
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDFS, RDF, OWL, XSD, SOSA, SKOS
from copy import deepcopy
from collections import defaultdict
from itertools import product

In [ ]:
model_path_middle = "./llama-2-7b-chat.Q4_K_M.gguf"
DEFAULT_ONTOLOGIES = ['sosa.ttl', 'rdfs.ttl', 'rdf.ttl', 'owl.ttl']
DEFAULT_IRIS = ['http://www.w3.org/ns/sosa', 'http://www.w3.org/2000/01/rdf-schema#', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'http://www.w3.org/2002/07/owl#']
get_iri = lambda x: '/'.join(x.split('/')[:-1])

def get_graph_data(graph):
  graph_data = graph.serialize()
  graph_data = re.sub("<[^ ]*\/([^ \/]*)>", "\g<1>", graph_data)
  graph_data = '\n'.join([x for x in graph_data.split('\n') if x[:7] != '@prefix' and x != ''])
  return graph_data

class GraphExplainer:
  def __init__(self, model_path=model_path_middle, n_ctx=4096, n_batch=2048):
    self.model_path = model_path
    self.n_threads = os.cpu_count()
    self.n_gpu = -1
    self.n_ctx = n_ctx
    self.n_batch = n_batch
    self.llm = self.load_model(n_ctx=self.n_ctx, n_batch=self.n_batch)

  def load_model(self, *args, **kwargs):
    return Llama(model_path=self.model_path, n_gpu_layers=self.n_gpu, n_threads=self.n_threads, *args, **kwargs)

  def explain_graph(self, system_prompt, user_prompt, *args, **kwargs):
    out = self.llm.create_chat_completion(
      messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      *args, **kwargs
    )
    return out['choices'][0]['message']['content']

  def set_ontologies_lookup(self, ontologies_paths, ontologies_iris):
    if type(ontologies_paths) == str:
      ontologies_paths = [ontologies_paths]
    if type(ontologies_iris) == str:
      ontologies_iris = [ontologies_iris]
    all_ontologies = deepcopy(DEFAULT_ONTOLOGIES)
    all_ontologies.extend(ontologies_paths)
    all_iris = deepcopy(DEFAULT_IRIS)
    all_iris.extend(ontologies_iris)

    self.ontologies_dict = {}
    for ont, iri in zip(all_ontologies, all_iris):
      ont_graph = Graph().parse(ont, format='ttl')
      self.ontologies_dict[iri]=ont_graph
    return

  def process_graph(self, file_path):
    processed_graph = Graph().parse(file_path, format='ttl')
    required_ontologies = []
    for s, p, o in processed_graph.triples((None, None, None)):
        for element in [s,p,o]:
          if 'http' in str(element):
            required_ontologies.append(element)
    required_ontologies = list(set(required_ontologies))

    ontologies_graph = Graph()
    for data_required in required_ontologies:
      temp_iri = get_iri(data_required)
      if temp_iri in self.ontologies_dict.keys():
        relevant_ontology = self.ontologies_dict[temp_iri]
        for s, p, o in relevant_ontology.triples((data_required, None, None)):
          ontologies_graph.add((s,p,o))
    ontology_text = get_graph_data(ontologies_graph)
    input_text = get_graph_data(processed_graph)
    return input_text, ontology_text

## Running the code

In [ ]:
explainer = GraphExplainer()
explainer.set_ontologies_lookup(['./content/pilot2.ttl'], ['https://assist-iot.eu/ontologies'])

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
cams = ['cam_0000003596.ttl', 'cam_0000005028.ttl', 'cam_0000014000.ttl', 'cam_0000014017.ttl', 'cam_0000014019.ttl', 'cam_0000014025.ttl', 'cam_0000014026.ttl', 'cam_0000014036.ttl']

## Results for cameras

In [ ]:
system_prompt_1 = lambda x: """
You are a master in the field of knowledge graphs. You perfectly know turtle and you explain their contents using layman's terms. Use the following ontology to understand the data.
""" + x
input_prompt_1 = lambda x: "Summarize only the data of the following graph, in sentences only. Focus on the values at the end of each line.\n\n" + x

In [ ]:
system_prompt_2 = lambda x: f"""
You are an expert in knowledge graphs. Use the following ontology to understand the data.

{x}

You are the assistant who, given a graph, can extract the most important information from it. Focus on the values at the end of each line.
Your answers should be clear, coherent and describe the most important facts recorded in the graph.
Please return full sentences, bullet points and focus on creating a human-readible text, and do not include class and property labels and names.
In summary, you should put focus on anomalies and potentially dangerous situations and clearly inform the user about them.
"""
input_prompt_2 = lambda x: "Summarise the data in the following graph.\n\n" + x

In [ ]:
for cam in cams:
  print(cam)
  input_text, ontology_text = explainer.process_graph(cam)
  print(explainer.explain_graph(system_prompt_1(ontology_text), input_prompt_1(input_text)))
  print()

cam_0000003596.ttl
  Here is a summary of the data in the graph, focusing on the values at the end of each line:
* "entrance_camera_1" is a sensor with an identifier "entrance_camera_1".
* The sensor made an observation, which resulted in a result with a label "Camera Result".
* The confidence of the method that made the decision is 99.99137115478516%.
* The image of the observation is stored in the Semantic Repository with a path "osh-camera/entrance_camera_1/32fec42735354991bfa27ba47b2a1758".
* The safety status of the workers is "status_unsafe", indicating that at least one worker in the group is not safe.
* The number of unsafe workers is 1.
* The result time of the observation is "2023-07-07T14:11:08.675910"^^xsd:dateTime.

cam_0000005028.ttl


Llama.generate: prefix-match hit


  * The graph represents a knowledge graph about construction site safety, with entities and properties defined using the SOSA ontology.
* An entrance camera (entrance_camera_1) is a sensor that made an observation on July 7, 2023, at 4:14:20 PM, with a result of "Camera result".
* The observation has a confidence level of 0.0 and an image associated with it.
* The image is stored in the Semantic Repository with a path of "osh-camera/entrance_camera_1/d3408b9fc7df4f83b7d8430bd22765e1".
* The observation has an indeterminate safety status, with 1 worker whose safety cannot be determined.
* There are no safe workers.
* The result time of the observation is "2023-07-07T16:14:20.406648"^^xsd:dateTime.

cam_0000014000.ttl


Llama.generate: prefix-match hit


  Sure! Here is a summary of the data in the graph, focusing on the values at the end of each line:
* "entrance_camera_1" is a sensor with an identifier "entrance_camera_1".
* The sensor made an observation, which resulted in a result.
* The result has a confidence level of 0.0.
* The result has an image associated with it, and the full path to the image in the Semantic Repository is "osh-camera/entrance_camera_1/c0a36e4d94a24dadac1e3bbd943da526".
* The sensor's safety status is "safe".
* There are 1 safe worker and 0 unsafe workers.
* The observation was made on July 14, 2023 at 1:00:49 PM (UTC).

cam_0000014017.ttl


...

  Sure! Here is a summary of the data in the graph, focusing on the values at the end of each line:
* "entrance_camera_1" is a sensor with an identifier "entrance_camera_1".
* The sensor made an observation, which resulted in a result.
* The result has a confidence level of 0.0.
* The result has an image, and its repository path is "osh-camera/entrance_camera_1/cbbcbf9c929c473484e2aee7f545b094".
* The safety status of the workers is indeterminate.
* There are 1 indeterminate worker and 3 safe workers.
* The result time of the observation is "2023-07-14T13:01:51.230969" (July 14, 2023, at 1:01:51 PM).

cam_0000014026.ttl


Llama.generate: prefix-match hit


  Sure! Here's a summary of the data in the graph, focusing on the values at the end of each line:
* "entrance_camera_1" is a sensor with an identifier.
* The sensor made an observation, which resulted in a result.
* The result has a confidence level of 0.0.
* The result also has an image, which is stored in the Semantic Repository with the path "osh-camera/entrance_camera_1/971d6d39a5854e2c83df8e64374822d0".
* The sensor's safety status is "safe".
* There are 4 safe workers.
* There are no unsafe workers.

cam_0000014036.ttl


Llama.generate: prefix-match hit


  Here is a summary of the data in the graph, focusing on the values at the end of each line:
* "entrance_camera_1" is a sensor with an identifier "entrance_camera_1".
* The sensor made an observation, which resulted in a result.
* The result has a confidence level of 0.0.
* The result has an image associated with it, and the full path to the image in the Semantic Repository is "osh-camera/entrance_camera_1/9c4275cfe9a54b02b248a80e74cc4ada".
* The sensor's safety status is "safe".
* There are 0 unsafe workers.
* There are 0 safe workers.



In [ ]:
for cam in cams:
  print(cam)
  input_text, ontology_text = explainer.process_graph(cam)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2(input_text)))
  print()

cam_0000003596.ttl


Llama.generate: prefix-match hit


  Based on the provided graph, here are the most important facts extracted:
* There is a sensor called "entrance_camera_1" with an identifier of "entrance_camera_1".
* The sensor has made an observation with a result called "CameraResult".
* The CameraResult has a confidence level of 99.99137115478516%.
* The CameraResult has an image associated with it, which can be found in the Semantic Repository with the path "osh-camera/entrance_camera_1/32fec42735354991bfa27ba47b2a1758".
* The sensor has indicated that there is at least one unsafe worker in the group, as the safety status is "ns1:status_unsafe".
* The number of workers whose safety cannot be determined is 0.
* The number of safe workers is also 0.
* The number of unsafe workers is 1.

cam_0000005028.ttl


Llama.generate: prefix-match hit


  Based on the provided graph, here are the most important facts extracted:
* There is a sensor called "entrance_camera_1" with an identifier of "entrance_camera_1".
* The sensor made an observation on July 7, 2023, at 4:14:20 PM (UTC) as indicated by the resultTime property.
* The observation has a result, which is a camera result with a confidence level of 0.0.
* The camera result has an image associated with it, and the image can be found in the Semantic Repository with the path "osh-camera/entrance_camera_1/d3408b9fc7df4f83b7d8430bd22765e1".
* The sensor has determined that there are 1 worker whose safety status is indeterminate.
* There are no safe workers or unsafe workers.
In summary, the graph indicates that there is a sensor called "entrance_camera_1" that made an observation on July 7, 2023, at 4:14:20 PM (UTC) with a confidence level of 0.0. The observation has a camera result with an image associated with it, and the sensor has determined that there is one worker whose safe

...

  Based on the provided graph, here are the most important facts extracted:
* There is a sensor called "entrance_camera_1" with an identifier of "entrance_camera_1".
* The sensor made an observation on July 14, 2023 at 13:01:51.230969 (UTC) and the result is a CameraResult.
* The confidence of the method that made the decision is 0.0.
* The image associated with this observation is stored in the Semantic Repository with the path "osh-camera/entrance_camera_1/cbbcbf9c929c473484e2aee7f545b094".
* The safety status of the workers is indeterminate, and there are 1 worker whose safety cannot be determined.
* The safe workers number is 3, and the unsafe workers number is 0.
In summary, there is a sensor called "entrance_camera_1" that made an observation on July 14, 2023 at 13:01:51.230969 (UTC) with a result of CameraResult, and the confidence of the method that made the decision is 0.0. The image associated with this observation is stored in the Semantic Repository with the path "osh-camer

Llama.generate: prefix-match hit


  Based on the provided graph, here are the most important facts extracted:
* There is a sensor named "entrance_camera_1" with an identifier of "entrance_camera_1".
* The sensor made an observation on July 14, 2023 at 1:01:53 PM (UTC) with a result.
* The result has a confidence level of 0.0.
* The result has an image associated with it.
* The image is stored in the Semantic Repository with a path of "osh-camera/entrance_camera_1/971d6d39a5854e2c83df8e64374822d0".
* The sensor has determined that all workers are safe.
* There are 4 safe workers.
* There are no unsafe workers.

It is important to note that the graph does not contain any anomalies or potentially dangerous situations, so there are no additional comments on that regard.

cam_0000014036.ttl


Llama.generate: prefix-match hit


  Based on the provided graph, here are the most important facts extracted:
* There is a sensor called "entrance_camera_1" with an identifier of "entrance_camera_1".
* The sensor made an observation on July 14, 2023, at 13:02:25.717797 UTC.
* The result of the observation is a camera result, which has a confidence level of 0.0.
* The camera result has an image associated with it, and the full path to the image in the Semantic Repository is "osh-camera/entrance_camera_1/9c4275cfe9a54b02b248a80e74cc4ada".
* The sensor has determined that all workers are safe.
* There are no unsafe workers.
* The number of safe workers is 0.

It appears that the sensor is monitoring the entrance of a site and has observed the safety status of the workers. The observation was made on a specific date and time, and the result is a camera image with a confidence level of 0.0. Additionally, the sensor has determined that all workers are safe, and there are no unsafe workers.



#### Explain to professor

In [ ]:
input_prompt_2_professor = lambda x: "I am a professor who specializes in knowledge graphs. Summarise the following graph focusing on its structure.\n\n" + x

In [ ]:
for cam in cams:
  print(cam)
  input_text, ontology_text = explainer.process_graph(cam)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2_professor(input_text)))
  print()

cam_0000003596.ttl


Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents information about various sensors, observations, results, and safety statuses. Here's a summary of the structure of the graph:
* The graph consists of 3 classes: `sosa:Sensor`, `sosa:Observation`, and `ns1:Image`.
* The `sosa:Sensor` class has 1 instance, `entrance_camera_1`, which is represented by a `ns2:identifier` property.
* The `sosa:Observation` class has 1 instance, which is represented by a `sosa:madeObservation` property that links to an `sosa:Result`.
* The `sosa:Result` class has 1 instance, which is represented by a `ns1:CameraResult` subproperty. This instance has properties such as `ns1:hasConfidence`, `ns1:hasImage`, `ns1:hasSafetyStatus`, and `ns1:indeterminateWorkersNum`.
* The `ns1:Image` class has 1 instance, which is represented by a `ns1:hasRepoPath` property that links to a specific path in the Semantic Repository.
* The graph also includes a `sosa:resultTime` property, which represents the time when the o

Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents information about various sensors, observations, results, and safety statuses. Here are the key points about the structure of the graph:
* There are several sensors (entrance_camera_1, entrance_camera_2, etc.) represented in the graph, each with its own identifier (ns2:identifier).
* Each sensor has made observations (sosa:madeObservation) for which it has a result (sosa:hasResult). The result is of type CameraResult.
* The result has a confidence level (ns1:hasConfidence) and an image (ns1:hasImage) associated with it. The image is stored in the Semantic Repository with a specific path (ns1:hasRepoPath).
* For entrance_camera_1, the safety status of the workers is indeterminate (ns1:hasSafetyStatus).
* The graph also includes information about the number of safe and unsafe workers (ns1:safeWorkersNum and ns1:unsafeWorkersNum) for each sensor.
* The result time for each observation is also provided (sosa:resultTime).
Overall, th

...

  The graph provided is a knowledge graph that represents information about various sensors, observations, results, and safety statuses. Here are the key points about the structure of the graph:
* The graph consists of 4 classes: sosa:Sensor, sosa:Observation, sosa:Result, and ns2:Image.
* There is one sensor (entrance_camera_1) that has made multiple observations (3).
* Each observation is associated with a result, which is either a camera result or a sample. The camera result has a confidence value of 0.0, and the sample has a result time of "2023-07-14T13:01:51.230969"^^xsd:dateTime.
* The sensor entrance_camera_1 has an indeterminate safety status, with 1 uncertain worker and 3 safe workers.
* The graph includes one image with a path to the Semantic Repository of "osh-camera/entrance_camera_1/cbbcbf9c929c473484e2aee7f545b094".
Overall, the graph suggests that there is a sensor (entrance_camera_1) at an entrance location, which has made observations and generated results. The safety

Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents information about a construction site, specifically the entrance camera #1. Here are some key points about the structure of the graph:
* The graph consists of three classes: Sensor (entrance_camera_1), Observation, and Result.
* The Sensor class (entrance_camera_1) has one instance (a single sensor) and is connected to two other classes: Observation and Result.
* The Observation class has one instance (an observation made by the entrance camera #1) and is connected to the Result class.
* The Result class has two instances (a result of the observation and a sample). Each result instance is connected to one observation and one sensor.
* The Observation class has three properties: hasResult, hasConfidence, and hasImage.
* The Result class has four properties: hasRepoPath, hasSafetyStatus, indeterminateWorkersNum, and safeWorkersNum.
* The hasResult property of the Observation class is connected to one Result instance.
* The hasConf

Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents information about a construction site, specifically the entrance camera #1. Here are some key points about the structure of the graph:
* The graph consists of three classes: Sensor (entrance_camera_1), Observation, and Result.
* The Sensor class has one instance, entrance_camera_1, which is represented by a NS2:identifier.
* The Observation class has two instances, each with a sosa:madeObservation property that links to a Result instance.
* Each Result instance has properties for confidence (0.0), image (a ns1:Image instance), and safety status (ns1:status_safe).
* The Observation instances also have properties for the time of the observation (sosa:resultTime with a xsd:dateTime value).
* There are no instances of the Actuation or Sampling classes.
* The graph does not contain any links to other classes or properties beyond those defined in the SOSA ontology.



In [ ]:
print(explainer.explain_graph(system_prompt_2, input_prompt_2_professor))

Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents information about various sensors, observations, results, and safety statuses. Here are the key points from the structure of the graph:
1. Entries: There are three entries in the graph - "entrance_camera_1", "entrance_camera_2", and "entrance_camera_3". Each entry represents a different camera at the entrance of a location.
2. Sensors: The graph has two sensors - "entrance_camera_1" and "entrance_camera_2". These sensors are responsible for making observations about the environment.
3. Observations: There are three observations made by the sensors - "observed_value_1", "observed_value_2", and "observed_value_3". Each observation represents a measurement or reading taken by the sensor at a particular time.
4. Results: The graph has two results - "result_1" and "result_2". These results are derived from the observations made by the sensors and represent the estimated value of a property of interest.
5. Confidence: The graph shows 

#### Explain to a contruction site supervisor

In [ ]:
input_prompt_2_supervisor = lambda x: "I am a supervisor at the contruction site. Report me about important incidents.\n\n" + input_text

In [ ]:
for cam in cams:
  print(cam)
  input_text, ontology_text = explainer.process_graph(cam)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2_supervisor(input_text)))
  print()

cam_0000003596.ttl


Llama.generate: prefix-match hit


  As the supervisor at the construction site, I have been informed that there is an important incident involving the entrance camera #1. According to the sensor data, the camera made an observation on July 7, 2023, at 2:11 PM, and the result was a safety status of "unsafe." The confidence level of the method that made the decision is 99.99137115478516%, and the image associated with this observation can be found in the Semantic Repository with the identifier "osh-camera/entrance_camera_1/32fec42735354991bfa27ba47b2a1758."
Additionally, there are 1 unsafe worker among the workers in the group. The unsafe worker's number is 1.
Please take appropriate action to address this incident and ensure the safety of all workers on the construction site.

cam_0000005028.ttl


Llama.generate: prefix-match hit


  There are several important incidents reported in the graph you provided:
* An unsafe situation has been detected at entrance camera 1. According to the observation, the confidence of the method that made the decision is 0.0, indicating a low level of certainty. Additionally, there are no safe workers in the group, and one worker's safety status is indeterminate.
* The number of workers whose safety cannot be determined is 1.
* The image associated with this observation is available at "osh-camera/entrance_camera_1/d3408b9fc7df4f83b7d8430bd22765e1".
It is important to take action to address these incidents, such as assigning a safety status to the worker whose safety is indeterminate and ensuring that all workers are safe.

cam_0000014000.ttl


...

  There are several important incidents reported in the graph you provided:
* An entrance camera (entrance_camera_1) has made an observation with a result of "ns1:CameraResult" and a confidence level of 0.0. This indicates that the method used to make the observation is not confident in its results.
* The observation was made on July 14, 2023 at 13:01:51.230969 UTC.
* There are 1 indeterminate worker whose safety status is "ns1:status_indeterminate".
* There are 3 safe workers.
* There are no unsafe workers.
In summary, there is one entrance camera that has made an observation with low confidence, and the observation was made on a specific date and time. Additionally, there are some workers whose safety status is unknown, but overall, there are more safe workers than unsafe workers. These incidents may indicate potential safety risks or issues that need to be addressed.

cam_0000014026.ttl


Llama.generate: prefix-match hit


  There are no major incidents or potential safety hazards reported in the given graph. However, there is an observation from a camera located at "entrance_camera_1" on July 14, 2023, at 13:01:53.157877 UTC. The observation has a result of "ns1:CameraResult" and the confidence in the method that made the decision is 0.0. The image associated with this observation can be found in the Semantic Repository with the path "osh-camera/entrance_camera_1/971d6d39a5854e2c83df8e64374822d0". Additionally, the safety status of the workers is "ns1:status_safe".

cam_0000014036.ttl


Llama.generate: prefix-match hit


  There are several important incidents reported in the graph you provided:
* An observation was made by entrance_camera_1 on July 14, 2023 at 13:02:25.717797 UTC, with a result of "ns1:CameraResult".
* The confidence of the method that made the decision is 0.0.
* There is an image associated with this observation, and it can be found in the Semantic Repository with the path "osh-camera/entrance_camera_1/9c4275cfe9a54b02b248a80e74cc4ada".
* The safety status of the workers is "ns1:status_safe", indicating that all workers are safe.
* The number of unsafe workers is 0, and the number of indeterminate workers is also 0.
* The result time of this observation is July 14, 2023 at 13:02:25.717797 UTC.
It is important to note that these incidents may require further investigation and action to ensure the safety of workers on the construction site.



In [ ]:
print(explainer.explain_graph(system_prompt_2, input_prompt_2_supervisor))

Llama.generate: prefix-match hit


  As the assistant, I have analyzed the provided knowledge graph and extracted the most important information regarding incidents at the construction site. Here are the key findings:
* There is a potential safety incident at the entrance of the site, as there is one unsafe worker (identified by "entrance_camera_1") with a confidence level of 99.99137115478516%.
* The unsafe worker has an image associated with their identity, which can be found in the Semantic Repository with the path "osh-camera/entrance_camera_1/32fec42735354991bfa27ba47b2a1758".
* The unsafe worker has a safety status of "ns1:status_unsafe", indicating that at least one worker in the group is not safe.
* The number of workers whose safety cannot be determined is 0, and the number of safe workers is also 0.
* The result time for this observation is "2023-07-07T14:11:08.675910"^^xsd:dateTime.
Based on these findings, it is important to take immediate action to address the safety incident at the entrance of the site. Th

### Results for tags

In [ ]:
input_text, ontology_text = explainer.process_graph('tags_0000003596.ttl')
explainer.explain_graph(system_prompt_1(ontology_text), input_prompt_1(input_text))

Llama.generate: prefix-match hit


'  * The Platform with identifier "tag-33298" hosts the Sensor with identifier "33298".\n* The Sensor made an Observation with a result of -4 for X, -3 for Y, and 256 for Z, and was recorded at time "2023-07-03T10:43:28.428205"^^xsd:dateTime.\n* The Sensor made another Observation with a result of -4 for X, -8 for Y, and 257 for Z, and was recorded at time "2023-07-03T10:43:29.428205"^^xsd:dateTime.\n* The Sensor also made a TagMetadataResult with an alarm of false, watch battery level of 90, and watch connected of true.\n* The BIMLocation has an accuracy value of 1 millimetre, and its location measurement type is "measurementNone". Its X, Y, and Z values are 0, 0, and 0 millimetres, respectively.'

In [ ]:
print(input_text)

33298 a sosa:Platform,
        ns2:IoTDevice ;
    ns3:identifier "tag-33298" ;
    sosa:hosts 33298 .
33298 a sosa:Sensor ;
    sosa:madeObservation [ a sosa:Observation ;
            sosa:hasResult [ a ns1:Measure,
                        sosa:Result,
                        aiot#MeasureResult ;
                    ns1:Unit ns2:bpm ;
                    ns1:hasNumericalValue 0 ],
                [ a sosa:Result,
                        ns2:AccelerationResult ;
                    ns2:accelerationWindow "2" ;
                    ns2:hasXValue [ a ns1:Measure ;
                            ns1:Unit ns2:tagAccelerationUnit ;
                            ns1:hasNumericalValue -4 ] ;
                    ns2:hasYValue [ a ns1:Measure ;
                            ns1:Unit ns2:tagAccelerationUnit ;
                            ns1:hasNumericalValue -3 ] ;
                    ns2:hasZValue [ a ns1:Measure ;
                            ns1:Unit ns2:tagAccelerationUnit ;
                         

In [ ]:
print(ontology_text)

sosa:Observation a rdfs:Class,
        owl:Class ;
    rdfs:label "Observation"@en ;
    rdfs:comment "Act of carrying out an (Observation) Procedure to estimate or calculate a value of a property of a FeatureOfInterest. Links to a Sensor to describe what made the Observation and how; links to an ObservableProperty to describe what the result is an estimate of, and to a FeatureOfInterest to detail what that property was associated with."@en ;
    rdfs:isDefinedBy sosa: ;
    skos:definition "Act of carrying out an (Observation) Procedure to estimate or calculate a value of a property of a FeatureOfInterest. Links to a Sensor to describe what made the Observation and how; links to an ObservableProperty to describe what the result is an estimate of, and to a FeatureOfInterest to detail what that property was associated with."@en ;
    skos:example "The activity of estimating the intensity of an Earthquake using the Mercalli intensity scale is an Observation as is measuring the moment mag